# brings new insights into Oracle's groups regarding localities, topics and events.

# important!: open neo4j, gather required libraries

In [27]:
import os
import csv
from neo4j import GraphDatabase, basic_auth

#currently neo4j is running on localhost
neo4jUrl = os.environ.get('NEO4J_URL',"bolt://localhost")
neo4jUser = os.environ.get('NEO4J_USER',"neo4j")
neo4jPass = os.environ.get('NEO4J_PASSWORD',"1")

driver = GraphDatabase.driver(neo4jUrl, auth=basic_auth(neo4jUser, neo4jPass))

graph = driver.session()

# query snippets

In [28]:
allNodes_where_person_rsvpd_to_oracle_and_others = "match (g:Group)-[:ORGANIZES]->(e:Event)<-[:RSVPd]-(p:Person)-[:RSVPd]->(e2:Event)<-[:ORGANIZES]-(g2:Group)  where g.name <> g2.name and g.name ='Code4Fun' or g.name='Swiss Data Circle'"

# List of Groups where people RSVP's rsvp'd to Swiss Data Circle or Code4Fun as well

In [32]:
def get_groups_where_rsvps_rsvpd_also_to_oracle(tx):
    for record in tx.run(allNodes_where_person_rsvpd_to_oracle_and_others +" return distinct g2.name"):
        print(str(record["g2.name"]))

with driver.session() as session:
    session.read_transaction(get_groups_where_rsvps_rsvpd_also_to_oracle)

Deep Learning Zurich (DLZH)
Power BI User Group Switzerland
Big Data Developers in Switzerland
Future of Data: Zürich
Zurich R User Group
AI for Intelligent Cloud Zürich
Swiss Big Data User Group
Zürich School of AI
PyData Zurich
AnalyticsCamp Switzerland
Pantalks - tech & non-tech talks @ Colab Zürich
Software Architecture and Project Design Zurich
Code4Fun
Voice Meetup Switzerland
Web Zürich
Oracle Beer
Swiss Oracle Cloud Community
Zürich Startup Founder 101


# Ranking of people who have rsvp'd at SwissDataCircle or Code4Fun and have rsvpd elsewhere.
Ranking is based on RSVP-count of the specific person

In [35]:
%%capture
def get_people_ranking(tx):
    for record in tx.run(allNodes_where_person_rsvpd_to_oracle_and_others+" return p.name as person_name, count(*) as rsvpd_event_amount order by rsvpd_event_amount desc"):
        print(str(record["rsvpd_event_amount"])  + "\t|" + str(record["person_name"]) )

with driver.session() as session:
    session.read_transaction(get_people_ranking)


# Ranking of people who just rsvp'd somewhere
Ranking is based on RSVP-count of the specific person

In [34]:
%%capture
def get_people_ranking(tx):
    for record in tx.run("match (g:Group)-[:ORGANIZES]->(e:Event)<-[r:RSVPd]-(p:Person) return p.name as person_name, count(r) as rsvpd_event_amount order by rsvpd_event_amount desc"):
        print(str(record["rsvpd_event_amount"])  + "\t|" + str(record["person_name"]) )

with driver.session() as session:
    session.read_transaction(get_people_ranking)


